1. if no bounding box is found return course bounding box

In [3]:
import sys
sys.path.append("/Users/ronald/Documents/code/harlus/ml/tools/harlus_chat")
from src.harlus_chat.build_graph import ChatAgentGraph

In [4]:
from doc_search import DocToolLoader
import nest_asyncio
nest_asyncio.apply()
toolloader = await DocToolLoader().load(
    file_path="/Users/ronald/Documents/code/harlus/ml/tools/harlus_chat/attributes/test1.pdf",
    file_name= "test1"
)
tool = toolloader.get()

Creating nodes for /Users/ronald/Documents/code/harlus/ml/tools/harlus_chat/attributes/test1.pdf ...
 - parsing PDF to JSON...
Started parsing the file under job_id 356e14eb-68da-4c78-9775-1f8e49d9a3f8
 - creating nodes from JSON...


100%|██████████| 73/73 [00:00<00:00, 58734.74it/s]


 - adding node relationships...
 - splitting tables...


  > splitting tables:   0%|          | 0/41 [00:00<?, ?it/s]





1it [00:00, 345.67it/s]
1it [00:00, 457.05it/s]
1it [00:00, 348.51it/s]
1it [00:00, 326.30it/s]




1it [00:00, 22192.08it/s]
1it [00:00, 1076.57it/s]
  > splitting tables: 100%|██████████| 41/41 [00:01<00:00, 27.10it/s]


 - adding metadata to nodes...
  > extracting metadata for SummaryExtractor ...


...

 - splitting text nodes...


  > splitting text nodes: 100%|██████████| 168/168 [00:00<00:00, 4828.80it/s]


 - adding file path to nodes...
getting single doc query engine from nodes
 - building vector index ...
 - building vector retriever ...
 - building keyword index ...
 - building summary index ...
 - building keyword retriever ...
 - building mix keyword vector retriever ...
 - building recursive keyword vector retriever ...
 - building mix keyword vector retriever query engine ...
 - building summary index query engine...
 - extracting metadata from query engines...


Extracting metadata: 100%|██████████| 7/7 [01:05<00:00,  9.34s/it]

 - building mix retriever query engine tool...


In [6]:
import json
gp = ChatAgentGraph([tool.to_langchain_tool()])
g =  gp.build()
async for chunk in gp.stream("Does apple have a business segment in the americas?"):
    try:
        chunk_data = chunk.split('\n')[0].split(': ', 1)[1]
        chunk_data = chunk_data.replace('data: ', '').replace('\n', '')
        chunk_data = chunk_data.split(':')[1]
        chunk_data = chunk_data.replace('}', '')
        chunk_data = chunk_data.replace('{', '')
        chunk_data = chunk_data.replace('"', '')
        print(chunk_data, end="", flush=True)

    except (IndexError, json.JSONDecodeError) as e:
        print(f"Error processing chunk: {e}")
        chunk_dict = {"data": {"text": ""}}

[harlus_chat] Streaming answer...
  Reading  [ Form   10 -K  for  the  Fiscal  Year  End ed  September   28 ,   202 4 ]  on  [ Apple  Inc .]  to  verify  if  [ Apple  has  a  business  segment  in  the  Americas ] ...                   Yes ,  Apple  has  a  business  segment  in  the  Americas ,  which  includes  both  North  and  South  America . [harlus_chat] Streaming source annotations...
[harlus_chat] Sent 2 source annotations
 4f3ce7cb-a72d-491f-989a-2034dd0e14ce, file_idError processing chunk: list index out of range


In [51]:
import fitz

def get_llamaparse_rects(file_path, retrieved_node, page_nb):
    doc = fitz.open(file_path)
    page = doc[page_nb]
    page_width = page.rect.width
    page_height = page.rect.height
    llamaparse_rects = retrieved_node.metadata.get("bounding_boxes")
    standard_rects = []
    for rect in llamaparse_rects:
        standard_rects.append({
                    "left": float(rect.get("x") / page_width) * 100,
                    "top": float(rect.get("y") / page_height) * 100,
                    "width": float(rect.get("w") / page_width) * 100,
                    "height": float(rect.get("h") / page_height) * 100,
                    "page": page_nb + 1 # 1-indexed convention 
                } )
    return standard_rects


    


In [52]:
from langchain_core.messages import (
    ToolMessage,
    SystemMessage,
    AIMessage,
    HumanMessage,
    AIMessageChunk,
)
from langgraph.graph import StateGraph, END, START
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.tools import BaseTool
from langgraph.graph.message import add_messages
from llama_index.core.tools import QueryEngineTool
from typing import Annotated, TypedDict, List, Iterator, AsyncIterator
from config import LLM, TAVILY_TOOL
import re
import json 
from rapidfuzz.fuzz import partial_ratio
from boundig_boxes import get_standard_rects_from_pdf, prune_overlapping_rects
from pydantic import BaseModel
import uuid


class ToolRetrievedNode(BaseModel):
    metadata: dict
    text: str

class BasicToolNode:
    """
    Runs the tools requested in the last AIMessage.

    Updates the state with the retrieved nodes from the tool calls.
    
    """

    def __init__(self, tools: list) -> None:
        self.tools_by_name = {tool.name: tool for tool in tools}

    def __call__(self, inputs: dict):

        # Get the last message
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No message found in input")
        outputs = []

        # Get the tool calls
        for tool_call in message.tool_calls:

            # Invoke the tool
            tool_result = self.tools_by_name[tool_call["name"]].invoke(tool_call["args"])

            # Try to get sources from the tool result
            has_retrieved_nodes = False
            try:
                retrieved_nodes_list = []
                for retrieved_node in tool_result.raw_output.source_nodes:
                    retrieved_nodes_list.append(ToolRetrievedNode(
                        metadata=retrieved_node.metadata,
                        text=retrieved_node.text
                    ))
                content = json.dumps(tool_result.content)
                has_retrieved_nodes = True
            except:
                content = json.dumps(tool_result)

            # Add the tool message to the outputs
            outputs.append(ToolMessage(
                content=content,
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
            ))
        
        # if there are retrieved nodes, add them to the state
        if has_retrieved_nodes:
            return {
                "messages": outputs,
                "retrieved_nodes": retrieved_nodes_list,
                "execution_plan_steps": inputs.get("execution_plan_steps", []),
                "current_step": inputs.get("current_step", "")
            }
        return {
            "messages": outputs,
            "retrieved_nodes": inputs.get("retrieved_nodes", []),
            "execution_plan_steps": inputs.get("execution_plan_steps", []),
            "current_step": inputs.get("current_step", "")
        }


class GraphState(TypedDict):
    messages: Annotated[list, add_messages]
    retrieved_nodes: list[list[any]]
    full_answer: str


class AsyncToolNode:
    def __init__(self, tools):
        self.tools = BasicToolNode(tools)

    async def __call__(self, state: GraphState) -> dict:
        return self.tools(state)

def sanitize_tool_name(name):
    return re.sub(r"[^a-zA-Z0-9_-]", "_", name)


class BoundingBox(BaseModel):
    left: float
    top: float
    width: float
    height: float
    page: int

class HighlightArea(BaseModel):
    bounding_boxes: list[BoundingBox]
    jump_to_page_number: int

class ChatSourceComment(BaseModel):
    id: str
    file_id: str
    thread_id: str
    message_id: str
    text: str
    highlight_area: HighlightArea
    next_chat_comment_id: str

class ChatAgentGraph:
    """
    ChatAgentGraph represents the agent behind the chat interface.
    
    This agent can be instantiated with a list of tools. After instantiating the agent, you must call the `build()` method.

    ```
    agent = ChatAgentGraph(tools=[tool1, tool2, tool3])
    agent.build()
    ```

    After building the agent, you can use the `stream()` method to start the chat. This method requires two arguments:
    - `user_message`: The message to send to the agent.
    - `thread_id`: The id of the thread to use. This is used to identify the thread in the database.

    ```
    async for message_chunk in agent.stream(user_message="Hello, how are you?", thread_id="1"):
        print(message_chunk)
    ```

    The `stream()` method acts as an EventSource stream. It outputs events with the following format:

    ```
    data: {"text": "Hello, how are you?"}
    event: "message"


    ```

    Currently, the stream supports the following events:
    - "message": A message from the agent. Data will have the following format:
    ```
    data: {"text": "Hello, how are you?"}
    event: "message"
    ```
    - "sources": A list of source annotations. Data will have the following format:
    ```
    data: [ChatSourceComment.model_dump(), ChatSourceComment.model_dump(), ...]
    event: "sources"
    ```
    - "complete": The stream is complete. Data will have the following format:
    ```
    data: "n.a."
    event: "complete"
    ```


    """
    def __init__(self, tools: List[any]):
        self.tools = []
        for tool in tools:
            if isinstance(tool, BaseTool):
                self.tools.append(tool)
            elif isinstance(tool, QueryEngineTool):
                self.tools.append(tool.to_langchain_tool())
            else:
                try:
                    self.tools.append(tool.to_langchain_tool())
                except:
                    raise ValueError(f"Tool {tool} is not a recognized tool.")
        for tool in self.tools:
            tool.name = sanitize_tool_name(tool.name)
        self.tools_descriptions_string = "\n - " + "\n -".join([f"{tool.name}: {tool.description}" for tool in tools])
        self.LLM = LLM
        self.TOOL_LLM = LLM.bind_tools(self.tools)
        self.graph = None
        self.config = {"configurable": {"thread_id": "1"}}

    async def _communicate_plan(self, state: GraphState) -> AsyncIterator[dict]:
        prompt = [
            SystemMessage(content=f"""
            You are an autonomous AI agent solving a task step-by-step using tools.
            Decide what to do next. YOU MUST BASE YOUR ANSWER ON THE TOOLS PROVIDED BELOW. DO NOT RELY ON PRIOR KNOWLEDGE.
                        
            WRITE A SHORT AND CONCISE PLAN LIKE. "Reading [Document Source] on [Company] from [date] to verify [Claim]..."
                        
            {self.tools_descriptions_string}
            """),
            *state["messages"], 
            HumanMessage(content="Provide a plan for your next step.")
        ]

        final = ""
        async for chunk in self.LLM.astream(prompt):
            delta = chunk.content or ""
            final += delta
        yield {
            "messages": state["messages"] + [AIMessage(content=final)],
            "retrieved_nodes": state.get("retrieved_nodes", []),
            "full_answer": state.get("full_answer", "") + final,
        }


    async def _call_tools(self, state: GraphState) -> AsyncIterator[dict]:
        prompt = [
            SystemMessage(content=f"""
            Only use the tools you have been provided with. 
            Base yourself on the plan provided by the user.
            If the plan does not require you to use any tools. Don't do anything.
                          
            ONLY USE THE TOOLS YOU HAVE BEEN PROVIDED WITH.
            """),
            *state["messages"],
        ]
        return {
            "messages": [await self.TOOL_LLM.ainvoke(prompt)],
            "retrieved_nodes": state["retrieved_nodes"],
            "full_answer": state["full_answer"],
        }

    @staticmethod
    def _custom_tools_condition(state: GraphState) -> str:
        last_msg = state["messages"][-1]
        if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
            return "tools"
        else:
            return "no_tools"


    async def _communicate_output(self, state: GraphState) -> AsyncIterator[dict]:
        prompt = [
            SystemMessage(content=f"""
            You will summarize an answer to the last Human Message based on the subsequent tool calls and AI messages
            """),
            *state["messages"]
        ]
        final = ""
        async for chunk in self.LLM.astream(prompt):
            delta = chunk.content or ""
            final += delta

        yield {
            "messages": state["messages"] + [AIMessage(content=final)],
            "retrieved_nodes": state.get("retrieved_nodes", []),
            "full_answer": state.get("full_answer", "") + final,
        }


    def build(self):

        # graph builder
        graph_builder = StateGraph(GraphState)

        # nodes
        graph_builder.add_node("communicate_plan", self._communicate_plan)
        graph_builder.add_node("call_tools", self._call_tools)
        graph_builder.add_node("tools", AsyncToolNode(tools=self.tools))

        # fixed edges
        graph_builder.add_edge(START, "communicate_plan")
        graph_builder.add_edge("communicate_plan", "call_tools")
        graph_builder.add_edge("tools", "call_tools")

        # conditional edges
        graph_builder.add_conditional_edges(
            "call_tools",
            self._custom_tools_condition,
            {"tools":"tools", "no_tools":END}
        )

        # compile
        graph = graph_builder.compile(checkpointer=MemorySaver())
        self.graph = graph

        return graph
    

    def _get_retrieved_nodes(self):

        # extract nodes which were retrieved during the last run through the graph
        retrieved_nodes = self.graph.get_state(self.config).values.get("retrieved_nodes", [])

        # prune nodes which have similar text
        pruned_retrieved_nodes = []
        for retrieved_node in retrieved_nodes:
            retrieved_node_text = retrieved_node.text.strip().lower()
            for pruned_retrieved_node in pruned_retrieved_nodes:
                pruned_retrieved_node_text = pruned_retrieved_node.text.strip().lower()
                if partial_ratio(retrieved_node_text, pruned_retrieved_node_text) > 90:
                    break
            else:
                pruned_retrieved_nodes.append(retrieved_node)
        
        return pruned_retrieved_nodes

    def _get_chat_source_comments(self):

        chat_source_comments = []

        # get the retrieved nodes from the graph
        retrieved_nodes = self._get_retrieved_nodes()
        nb_messages = len(self.graph.get_state(self.config).values.get("messages", []))

        # convert the retrieved nodes to source annotations
        last_unique_id = ""
        for retrieved_node in retrieved_nodes:
            file_path = retrieved_node.metadata.get("file_path")
            text = retrieved_node.text
            page_nb = retrieved_node.metadata.get("page_nb")

            # get the rects from the llama parse tool (course)
            backup_standard_rects = get_llamaparse_rects(file_path, retrieved_node, page_nb)

            # get the rects based on text from the pdf (fine-grained)
            standard_rects = get_standard_rects_from_pdf(file_path, text, page_nb)
            standard_rects = prune_overlapping_rects(standard_rects)

            # if no fine-grained rects are found, use the course rects
            if len(standard_rects) == 0:
                print("[harlus_chat] No fine-grained rects found, using course rects")
                standard_rects = backup_standard_rects

            # convert to ChatSourceComment framework
            unique_id = str(uuid.uuid4())
            bboxes = [BoundingBox(**rect) for rect in standard_rects]
            highlight_area = HighlightArea(bounding_boxes=bboxes, jump_to_page_number=page_nb)
            chat_source_comment = ChatSourceComment(
                highlight_area=highlight_area,
                id=unique_id,
                file_id=file_path,
                thread_id=self.config["configurable"].get("thread_id"),
                message_id=str(nb_messages),
                text="Response source",
                next_chat_comment_id=last_unique_id
            )
            last_unique_id = unique_id
            chat_source_comments.append(chat_source_comment)

        return chat_source_comments
    

    async def stream(self, user_message: str, thread_id: str = "1"):

        input_state = {
            "messages": [("user", user_message)], 
            "retrieved_nodes": [], 
            "full_answer": ""
        }

        self.config["configurable"]["thread_id"] = thread_id

        # 1. stream the answer 
        print("[harlus_chat] Streaming answer...")
        async for message_chunk, metadata in self.graph.astream(
            input_state,
            stream_mode="messages",
            config = self.config
        ):
            try:
                if isinstance(message_chunk, AIMessageChunk):
                    response = '\n'.join([
                        f'data: {json.dumps({"text": message_chunk.content})}',
                        f'event: {"message"}',
                        '\n\n'
                    ])
                
                    yield response
            except Exception as e:
                print(f"Streaming error: {e}")
            
        # 2. stream the source annotations
        print("[harlus_chat] Streaming source annotations...")
        try:
            data = self._get_chat_source_comments()
            data = [d.model_dump() for d in data]
            response = '\n'.join([
                    f'data: {json.dumps(data)}',
                    f'event: {"sources"}',
                    '\n\n'
            ])
            print(f"[harlus_chat] Sent {len(data)} source annotations")
            yield response
        except Exception as e:
            print(f"[harlus_chat] Error sending source annotations: {e}")

        # 3. stream the completion
        response = '\n'.join([
                    f'data: ',
                    f'event: {"complete"}',
                    '\n\n'
            ])
        yield response
    
    


Creating nodes for /Users/ronald/Documents/code/harlus/ml/tools/harlus_chat/attributes/test1.pdf ...
 - parsing PDF to JSON...
Started parsing the file under job_id c655d5e7-2348-47f0-bef3-c6ccb31ff767
 - creating nodes from JSON...


100%|██████████| 73/73 [00:00<00:00, 26250.36it/s]


 - adding node relationships...
 - splitting tables...


  > splitting tables:   0%|          | 0/41 [00:00<?, ?it/s]














1it [00:00, 237.01it/s]
1it [00:00, 257.07it/s]
1it [00:00, 263.79it/s]
1it [00:00, 166.86it/s]
1it [00:00, 79.66it/s]
1it [00:00, 70.59it/s]
  > splitting tables: 100%|██████████| 41/41 [00:01<00:00, 28.19it/s]


 - adding metadata to nodes...
  > extracting metadata for SummaryExtractor ...


...

 - splitting text nodes...


  > splitting text nodes: 100%|██████████| 168/168 [00:00<00:00, 5628.14it/s]


 - adding file path to nodes...
getting single doc query engine from nodes
 - building vector index ...
 - building vector retriever ...
 - building keyword index ...
 - building summary index ...
 - building keyword retriever ...
 - building mix keyword vector retriever ...
 - building recursive keyword vector retriever ...
 - building mix keyword vector retriever query engine ...
 - building summary index query engine...
 - extracting metadata from query engines...


Extracting metadata: 100%|██████████| 7/7 [00:16<00:00,  2.34s/it]

 - building mix retriever query engine tool...


[harlus_chat] Streaming answer...
  Reading  the  Form   10 -K  for  Apple  Inc .  for  the  fiscal  year  ended  September   28 ,   202 4 ,  to  verify  if  Apple  has  a  business  segment  in  the  Americas .                   Yes ,  Apple  has  a  business  segment  in  the  Americas ,  which  includes  both  North  and  South  America . [harlus_chat] Streaming source annotations...
[harlus_chat] Sent 1 source annotations
 9ea835cc-03ee-4b01-987a-1870556de6a2, file_idError processing chunk: list index out of range


In [56]:
comments = gp._get_chat_source_comments()

In [57]:
comments

[ChatSourceComment(id='5c678fca-e1f2-428f-b04e-734824db2f1b', file_id='/Users/ronald/Documents/code/harlus/ml/tools/harlus_chat/attributes/test1.pdf', thread_id='1', message_id='5', text='Response source', highlight_area=HighlightArea(bounding_boxes=[BoundingBox(left=8.20625241063222, top=56.32648603933158, width=83.58697073800224, height=0.9763196641645636, page=5), BoundingBox(left=8.20625241063222, top=57.46550933765402, width=83.58691944795497, height=0.9763196641645636, page=5), BoundingBox(left=8.20625241063222, top=58.68592250941768, width=83.58710409212513, height=0.9763196641645636, page=5), BoundingBox(left=8.20625241063222, top=59.82494580774013, width=83.5875554445411, height=0.9763196641645636, page=5), BoundingBox(left=8.20625241063222, top=60.964001725801666, width=83.58652964359572, height=0.9763160397491092, page=5), BoundingBox(left=8.20625241063222, top=62.18438590224169, width=74.93465263302586, height=0.9763160397491092, page=5)], jump_to_page_number=5), next_chat_